In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%env XLA_PYTHON_CLIENT_MEM_FRACTION=0.95

In [ ]:
from pathlib import Path

import jax.numpy as jnp
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import neural_tangents as nt

from idiots.dataset.dataloader import DataLoader
from idiots.experiments.grokking.training import restore, eval_step
from idiots.utils import metrics

In [ ]:
# checkpoint_dir = Path("/home/dc755/idiots/logs/grokking/exp22/checkpoints")
checkpoint_dir = Path("logs/grokking/exp21/checkpoints")

# def linear_model_from(step):
config, state, ds_train, ds_test = restore(checkpoint_dir, 0)
kernel_fn = nt.empirical_ntk_fn(state.apply_fn, trace_axes=(), vmap_axes=0)
kernel_fn = nt.batch(kernel_fn, batch_size=32)

In [ ]:
train_size = 512
# test_size = 128

x_train = ds_train["x"][:train_size]
# x_test = ds_test["x"][:test_size]

k_train_train = kernel_fn(x_train, x_train, state.params)
k_train_train.shape, k_train_train.size, k_train_train.nbytes / 1e6, k_train_train.dtype

In [ ]:
plt.imshow(jnp.abs(k_train_train[0, 4]))
plt.colorbar()

In [ ]:
df = pd.DataFrame(data)

df_loss = df[["step", "train_loss", "test_loss"]]
df_loss = df_loss.melt("step", var_name="split", value_name="loss")
df_loss["split"] = df_loss["split"].str.replace("_loss", "")


df_acc = df[["step", "train_acc", "test_acc"]]
df_acc = df_acc.melt("step", var_name="split", value_name="accuracy")
df_acc["split"] = df_acc["split"].str.replace("_acc", "")

fig, axs = plt.subplots(1, 2, figsize=(12, 6))

sns.lineplot(data=df_loss, x="step", y="loss", hue="split", marker="o", ax=axs[0])
sns.lineplot(data=df_acc, x="step", y="accuracy", hue="split", marker="o", ax=axs[1])